In [1]:
# Importing pandas 
import pandas as pd 
# loading the clean data dataset
# Installed openpyxl to open excel files in pandas 
# !pip install openpyxl

clean_data = pd.read_excel('Clean data.xlsx')
clean_data.head()

,Policy No,Ben Ord No,Status,Plan No,Issue Age,Issue Date,Next Prem Date,Sex,Sum Assured,Std Prem,Extra Prem,Frequency,Fund Value,Maturity Date,Term,WOP,Ben Code No,Accumulated Guarantee,Category
0,123-3,1,A,123,41,2003-08-02,2023-12-02,F,1000000.0,90996.00,0.0,M,0.0,2028-08-02 00:00:00,25,0,41,0.0,Money Back
1,122-459,1,A,122,40,2004-01-01,2024-01-01,M,300000.0,29220.00,0.0,M,0.0,2024-01-01 00:00:00,20,0,44,0.0,Education
2,122-631,1,A,122,30,2004-01-01,2024-01-01,M,300000.0,27528.00,0.0,M,0.0,2024-01-01 00:00:00,20,0,44,0.0,Education
3,120-137,1,A,120,32,2003-10-01,2024-02-01,F,350000.0,31008.00,0.0,M,0.0,2028-10-01 00:00:00,25,0,40,0.0,Money Back
4,121-159,1,A,121,33,2003-11-01,2024-02-01,F,320000.0,29776.16,0.0,Q,0.0,2027-11-01 00:00:00,24,0,42,0.0,Money Back


In [2]:
# Taking a look at the column names 
clean_data.columns

Index(['Policy No', 'Ben Ord No', 'Status', 'Plan No', 'Issue Age',
       'Issue Date', 'Next Prem Date', 'Sex', 'Sum Assured', 'Std Prem',
       'Extra Prem', 'Frequency', 'Fund Value', 'Maturity Date', 'Term', 'WOP',
       'Ben Code No', 'Accumulated Guarantee', 'Category'],
      dtype='object')

In [3]:
# Loading the death data
data_death = pd.read_excel('Death data.xlsx')
# Viewing the loaded death data
data_death.head()

,POLICY_NO,BENF_ORD,POL_STAT,PLANNO,ISSUE_AGE_LIFE1,START_DATE,NX_EXP_PR_DT,SEX_LIFE1,SEX_LIFE2,SEX_LIFE21,...,FREQ,END_DATE,TERM,BENFNO,DOB_LIFE1,DOB_LIFE2,DATE_OF_DEATH,NOTICE_DATE,CLAIM_TYPE,REASON_CODE1
0,126-42,1,M,126,5,2003-12-01,2017-12-01,M,NaN,X,...,1,2017-12-01,14,46,1999-02-25,NaN,2013-10-30,2015-01-19,OP,NaN
1,121-46,1,L,121,47,2003-09-01,2013-10-01,M,NaN,X,...,12,2019-09-01,16,42,1957-08-07,NaN,2014-12-15,2015-01-30,X,NaN
2,122-483,1,M,122,37,2004-01-01,2017-01-01,F,NaN,X,...,1,2017-01-01,13,44,1967-12-14,NaN,2014-08-11,2014-08-27,PD,NaN
3,1840652,1,M,109,56,1994-10-01,2004-10-01,F,NaN,X,...,1,2004-10-01,10,64,1938-12-10,NaN,2009-04-22,2009-05-26,X,NaN
4,1898630,1,M,109,35,2000-05-01,2012-05-01,M,NaN,X,...,1,2012-05-01,12,64,1965-11-08,NaN,2009-02-06,2009-06-15,PD,NaN


In [4]:
# Loading the dataframe containing the expected mortality rates for each age
expected_mortality_df = pd.read_excel('Mortality Tables.xlsx')
# Taking a look at the loaded dataframe and its columns
expected_mortality_df.head()

,Age Last Birthday,KE 01-03,KE 07-10
0,0,3.121,2.3005
1,1,3.121,2.3005
2,2,3.121,2.3005
3,3,3.121,2.3005
4,4,3.121,2.3005


In [5]:
# Creating a function that will create other computed columns
def compute(data,death_data):
    
#     Converting certain columns to datetime objects
    data['Issue Date'] = pd.to_datetime(data['Issue Date'])
    data['Next Prem Date'] = pd.to_datetime(data['Next Prem Date'])
    data['Maturity Date'] = pd.to_datetime(data['Maturity Date'],errors = 'coerce')
    
#     Adding a new column containing the Issue year
    data['Issue Year'] =  data['Issue Date'].dt.year
    
#     Adding a new column of next premium
    data['Next Prem Year'] = data['Next Prem Date'].dt.year
    
#     Creating a date of birth column 
#     data['Date of Birth'] = data['Issue Date'] - (data['Issue Age'].astype(int)*365.25).astype('timedelta64[D]')
    data['Date of Birth'] = data['Issue Date'] - pd.to_timedelta(data['Issue Age'] * 365.25, unit='D')
    data['Date of Birth'] = data['Date of Birth'].dt.date
    
# Creating a column indicating date of death
    data['Date of Death'] = [death_data.loc[death_data['POLICY_NO'] == item, 'DATE_OF_DEATH'].iloc[0]
                               if item in death_data['POLICY_NO'].values else "" 
                               for item in data['Policy No']]
    
#     Converting the created column to a datetime object
    data['Date of Death'] =  pd.to_datetime(data['Date of Death'],errors = 'coerce')
    
#     Creating a column containing the year of death
    data['Year of Death'] = data['Date of Death'].dt.year
    
#     Creating a column containing the notice date
    data['Notice Date'] = [death_data.loc[death_data['POLICY_NO'] == item, 'NOTICE_DATE'].iloc[0]
                               if item in death_data['POLICY_NO'].values else "" 
                               for item in data['Policy No']]
    
#     Converting the created column to a datetime object
    data['Notice Date'] = pd.to_datetime(data['Notice Date'])
    
#     Creating a new column showing the reporting delay in months
    data['Reporting Delay_Months'] = (data['Notice Date']  - \
                                      data['Date of Death']).astype('timedelta64[M]') 
    
# Creating a series containing the date to compare
    x = pd.Series([pd.to_datetime('2013-1-1',errors = 'coerce')]*len(data))
# adding the series to data
    data['Date_time_series'] = x
# Creating the entry date column
    data['Entry Date'] = data[['Issue Date','Date_time_series']].max(axis=1)
# Dropping the Date_time_series column
    data.drop(['Date_time_series'],axis=1,inplace = True)
# Importing numpy 
    import numpy as np 
# Creating a series containing the date to compare
    x_1 = pd.Series([pd.to_datetime('2022-12-31',errors = 'coerce')]*len(data))
# adding the series to data
    data['Date_time'] = x_1
# Creating a column containing the mininum valu of four columns
    data['Min_1'] = data[['Date of Death','Date_time','Maturity Date','Next Prem Date']].min(axis=1)
# Creating the exit date column
    data['Exit Date'] = np.where(data['Min_1'] < data['Entry Date'], data['Entry Date'], data['Min_1'])
# Dropping the Date_time column
    data.drop(['Date_time','Min_1'],axis=1,inplace = True)
# Creating a column indicating death
    data['Death Indicator'] = data['Date of Death'].apply(lambda x: 0 if x == pd.NaT else 1)
    
# Creating a column showing age at entry in years
# Converting date of birth to a datetime object
    data['Date of Birth'] = pd.to_datetime(data['Date of Birth'])
    data['Age @entry-Years'] = np.floor((data['Entry Date']-data['Date of Birth']).dt.days/365.25)
    
# Creating a column showing age at exit
    data['Age @exit-Years'] = np.floor((data['Exit Date']-data['Date of Birth']).dt.days/365.25)
# Creating a new column Exposure Contribution to Entry Age(in_years)
    data['Exposure Contribution to Entry Age(in_years)'] = np.where(data['Age @exit-Years'] == data['Age @entry-Years'],
                                                                   (data['Exit Date']-data['Entry Date']).dt.days/365.25,
                                                                   1-((data['Entry Date'] - data['Date of Birth']).dt.days / 365.25
                                                                   - np.floor((data['Entry Date'] - data['Date of Birth']).dt.days / 365.25)))
# Creating a new column Exposure Contribution to Exit Age(in_years)
    data['Exposure Contribution to Exit Age(in_years)'] = np.where(data['Age @exit-Years'] == data['Age @entry-Years'],
                                                                   0,
                                                                   (data['Exit Date'] - data['Date of Birth']).dt.days / 365.25
                                                                   - np.floor((data['Exit Date'] - data['Date of Birth']).dt.days / 365.25))
    
# Creating a new column indicating the no of deaths
    data['No_of death'] = data['Year of Death'].apply(lambda x: 1 if x >= 2013 and x <= 2022 else 0 )
    
#     Creating a column indicating whether male or female
    data['male(1)/female(0)_indicator'] = data['Sex'].apply(lambda x: 1 if x == 'M' else 0) 
    
#     Creating a new column Business Type
    data['Business Type'] = data['Category'].apply(lambda x:'Unit Linked' if x in 
                                                   ['Imarika','Other UL','SupercashBack '] 
                                                   else 'Conventional' )
    
#     Creating a columns indicating Lien or Non lien
    data['Lien/Non_Lien'] = data['Plan No'].apply(lambda x: 'Lien'if x in [120,121,122,137,138,148] 
                                                 else 'Non Lien')
    
#     Creating a column indicating all polocies
    data['All Policies Indicator'] =  1
    
#     Creating a column containing the total premium
    data['Total Premium'] = data['Std Prem'] + data['Extra Prem']
    
#     Converting the date of death column in death_data to a datetime object
    death_data['DATE_OF_DEATH'] = pd.to_datetime(death_data['DATE_OF_DEATH'],errors='coerce')
#     Creating a column indicating year of death in the death_data
    death_data['Year of Death'] = death_data['DATE_OF_DEATH'].dt.year
    
#     Creating a column indicating the Business Type
    death_data['Business Type'] = [data.loc[data['Policy No'] == item, 'Business Type'].iloc[0]
                                   if item in data['Policy No'].values else "" 
                                   for item in death_data['POLICY_NO']]

#     Creating a column indicating Lien/Non Lien
    death_data['Lien/Non Lien'] = [data.loc[data['Policy No'] == item, 'Lien/Non_Lien'].iloc[0]
                                   if item in data['Policy No'].values else "Lien" 
                                   for item in death_data['POLICY_NO']]
#     Creating a column indicating the gender
    death_data['Gender'] = [data.loc[data['Policy No'] == item, 'Sex'].iloc[0]
                                   if item in data['Policy No'].values else "M" 
                                   for item in death_data['POLICY_NO']]
    
#     Converting the NOTICE_DATE column to a datetime object
    death_data['NOTICE_DATE'] = pd.to_datetime(death_data['NOTICE_DATE'],errors='coerce')
#     Creating a column indicating the reporting delay(in months) in death_data
    death_data['Reporting Delay'] = np.round((death_data['NOTICE_DATE'] \
                                              - death_data['DATE_OF_DEATH']).dt.days/29.6666667)
    
    return data,death_data
    


In [6]:
data,death_data = compute(clean_data,data_death)
data.head()

,Policy No,Ben Ord No,Status,Plan No,Issue Age,Issue Date,Next Prem Date,Sex,Sum Assured,Std Prem,...,Age @entry-Years,Age @exit-Years,Exposure Contribution to Entry Age(in_years),Exposure Contribution to Exit Age(in_years),No_of death,male(1)/female(0)_indicator,Business Type,Lien/Non_Lien,All Policies Indicator,Total Premium
0,123-3,1,A,123,41,2003-08-02,2023-12-02,F,1000000.0,90996.00,...,50.0,60.0,0.579740,0.416153,0,0,Conventional,Non Lien,1,90996.00
1,122-459,1,A,122,40,2004-01-01,2024-01-01,M,300000.0,29220.00,...,49.0,58.0,0.997947,0.997947,0,1,Conventional,Lien,1,29220.00
2,122-631,1,A,122,30,2004-01-01,2024-01-01,M,300000.0,27528.00,...,39.0,48.0,0.996578,0.999316,0,1,Conventional,Lien,1,27528.00
3,120-137,1,A,120,32,2003-10-01,2024-02-01,F,350000.0,31008.00,...,41.0,51.0,0.746064,0.249829,0,0,Conventional,Lien,1,31008.00
4,121-159,1,A,121,33,2003-11-01,2024-02-01,F,320000.0,29776.16,...,42.0,52.0,0.828884,0.167009,0,0,Conventional,Lien,1,29776.16


In [7]:
death_data.head(10)

,POLICY_NO,BENF_ORD,POL_STAT,PLANNO,ISSUE_AGE_LIFE1,START_DATE,NX_EXP_PR_DT,SEX_LIFE1,SEX_LIFE2,SEX_LIFE21,...,DOB_LIFE2,DATE_OF_DEATH,NOTICE_DATE,CLAIM_TYPE,REASON_CODE1,Year of Death,Business Type,Lien/Non Lien,Gender,Reporting Delay
0,126-42,1,M,126,5,2003-12-01,2017-12-01,M,NaN,X,...,NaN,2013-10-30,2015-01-19,OP,NaN,2013,Conventional,Non Lien,M,15.0
1,121-46,1,L,121,47,2003-09-01,2013-10-01,M,NaN,X,...,NaN,2014-12-15,2015-01-30,X,NaN,2014,Conventional,Lien,M,2.0
2,122-483,1,M,122,37,2004-01-01,2017-01-01,F,NaN,X,...,NaN,2014-08-11,2014-08-27,PD,NaN,2014,Conventional,Lien,F,1.0
3,1840652,1,M,109,56,1994-10-01,2004-10-01,F,NaN,X,...,NaN,2009-04-22,2009-05-26,X,NaN,2009,,Lien,M,1.0
4,1898630,1,M,109,35,2000-05-01,2012-05-01,M,NaN,X,...,NaN,2009-02-06,2009-06-15,PD,NaN,2009,,Lien,M,4.0
5,1926701,1,M,109,39,2002-03-01,2017-03-01,M,NaN,X,...,NaN,2009-09-20,2010-08-26,PD,NaN,2009,Conventional,Non Lien,M,11.0
6,20010663,1,X,108,34,1990-06-01,2009-05-01,M,NaN,X,...,NaN,2009-03-26,2009-05-05,X,NaN,2009,,Lien,M,1.0
7,20046757,1,M,106,3,1996-09-01,2013-09-01,F,NaN,X,...,NaN,2005-11-01,2014-04-28,OP,NaN,2005,Conventional,Non Lien,F,104.0
8,20051277,1,X,108,37,1997-12-01,2016-10-01,F,NaN,X,...,NaN,2016-09-21,2016-10-17,X,NaN,2016,Conventional,Non Lien,F,1.0
9,20053010,1,X,108,32,1997-08-01,2003-01-01,F,NaN,X,...,NaN,2002-09-23,2012-09-05,X,NaN,2002,,Lien,M,123.0


In order to create the summary tables, a column containing the specific age bands will be created.The dataset from the clean_data will be used.

In [8]:
# Converting the specific column to integer type
data['Age @exit-Years'] = data['Age @exit-Years'].astype(int)
data['Age @exit-Years'].dtype
data.head()

,Policy No,Ben Ord No,Status,Plan No,Issue Age,Issue Date,Next Prem Date,Sex,Sum Assured,Std Prem,...,Age @entry-Years,Age @exit-Years,Exposure Contribution to Entry Age(in_years),Exposure Contribution to Exit Age(in_years),No_of death,male(1)/female(0)_indicator,Business Type,Lien/Non_Lien,All Policies Indicator,Total Premium
0,123-3,1,A,123,41,2003-08-02,2023-12-02,F,1000000.0,90996.00,...,50.0,60,0.579740,0.416153,0,0,Conventional,Non Lien,1,90996.00
1,122-459,1,A,122,40,2004-01-01,2024-01-01,M,300000.0,29220.00,...,49.0,58,0.997947,0.997947,0,1,Conventional,Lien,1,29220.00
2,122-631,1,A,122,30,2004-01-01,2024-01-01,M,300000.0,27528.00,...,39.0,48,0.996578,0.999316,0,1,Conventional,Lien,1,27528.00
3,120-137,1,A,120,32,2003-10-01,2024-02-01,F,350000.0,31008.00,...,41.0,51,0.746064,0.249829,0,0,Conventional,Lien,1,31008.00
4,121-159,1,A,121,33,2003-11-01,2024-02-01,F,320000.0,29776.16,...,42.0,52,0.828884,0.167009,0,0,Conventional,Lien,1,29776.16


In [9]:
# Creating a function to create the age bands
def create_ageband(age):
    if age <= 25:
        return '0-25'
    elif age <= 30:
        return '26-30'
    elif age <= 35:
        return '31-35'
    elif age <= 40:
        return '36-40'
    elif age <= 45:
        return '41-45'
    elif age <= 50:
        return '46-50'
    elif age <= 55:
        return '51-55'
    else:
        return 'Above 55'

In [10]:
# Creating the agebands for age @ exit years
data['Age_band'] = data['Age @exit-Years'].apply(create_ageband)
data.head()

,Policy No,Ben Ord No,Status,Plan No,Issue Age,Issue Date,Next Prem Date,Sex,Sum Assured,Std Prem,...,Age @exit-Years,Exposure Contribution to Entry Age(in_years),Exposure Contribution to Exit Age(in_years),No_of death,male(1)/female(0)_indicator,Business Type,Lien/Non_Lien,All Policies Indicator,Total Premium,Age_band
0,123-3,1,A,123,41,2003-08-02,2023-12-02,F,1000000.0,90996.00,...,60,0.579740,0.416153,0,0,Conventional,Non Lien,1,90996.00,Above 55
1,122-459,1,A,122,40,2004-01-01,2024-01-01,M,300000.0,29220.00,...,58,0.997947,0.997947,0,1,Conventional,Lien,1,29220.00,Above 55
2,122-631,1,A,122,30,2004-01-01,2024-01-01,M,300000.0,27528.00,...,48,0.996578,0.999316,0,1,Conventional,Lien,1,27528.00,46-50
3,120-137,1,A,120,32,2003-10-01,2024-02-01,F,350000.0,31008.00,...,51,0.746064,0.249829,0,0,Conventional,Lien,1,31008.00,51-55
4,121-159,1,A,121,33,2003-11-01,2024-02-01,F,320000.0,29776.16,...,52,0.828884,0.167009,0,0,Conventional,Lien,1,29776.16,51-55


In [11]:
# Creating a column containing the Age band index
def ageband_index(age_band):
    if age_band == '0-25':
        return 1
    elif age_band == '26-30':
        return 2
    elif age_band == '31-35':
        return 3
    elif age_band == '36-40':
        return 4
    elif age_band == '41-45':
        return 5
    elif age_band == '46-50':
        return 6
    elif age_band == '51-55':
        return 7
    else:
        return 8
        

In [12]:
# Creating the ageband indices
data['Age_band_index'] = data['Age_band'].apply(ageband_index)
data.head()

,Policy No,Ben Ord No,Status,Plan No,Issue Age,Issue Date,Next Prem Date,Sex,Sum Assured,Std Prem,...,Exposure Contribution to Entry Age(in_years),Exposure Contribution to Exit Age(in_years),No_of death,male(1)/female(0)_indicator,Business Type,Lien/Non_Lien,All Policies Indicator,Total Premium,Age_band,Age_band_index
0,123-3,1,A,123,41,2003-08-02,2023-12-02,F,1000000.0,90996.00,...,0.579740,0.416153,0,0,Conventional,Non Lien,1,90996.00,Above 55,8
1,122-459,1,A,122,40,2004-01-01,2024-01-01,M,300000.0,29220.00,...,0.997947,0.997947,0,1,Conventional,Lien,1,29220.00,Above 55,8
2,122-631,1,A,122,30,2004-01-01,2024-01-01,M,300000.0,27528.00,...,0.996578,0.999316,0,1,Conventional,Lien,1,27528.00,46-50,6
3,120-137,1,A,120,32,2003-10-01,2024-02-01,F,350000.0,31008.00,...,0.746064,0.249829,0,0,Conventional,Lien,1,31008.00,51-55,7
4,121-159,1,A,121,33,2003-11-01,2024-02-01,F,320000.0,29776.16,...,0.828884,0.167009,0,0,Conventional,Lien,1,29776.16,51-55,7


In [13]:
data.columns

Index(['Policy No', 'Ben Ord No', 'Status', 'Plan No', 'Issue Age',
       'Issue Date', 'Next Prem Date', 'Sex', 'Sum Assured', 'Std Prem',
       'Extra Prem', 'Frequency', 'Fund Value', 'Maturity Date', 'Term', 'WOP',
       'Ben Code No', 'Accumulated Guarantee', 'Category', 'Issue Year',
       'Next Prem Year', 'Date of Birth', 'Date of Death', 'Year of Death',
       'Notice Date', 'Reporting Delay_Months', 'Entry Date', 'Exit Date',
       'Death Indicator', 'Age @entry-Years', 'Age @exit-Years',
       'Exposure Contribution to Entry Age(in_years)',
       'Exposure Contribution to Exit Age(in_years)', 'No_of death',
       'male(1)/female(0)_indicator', 'Business Type', 'Lien/Non_Lien',
       'All Policies Indicator', 'Total Premium', 'Age_band',
       'Age_band_index'],
      dtype='object')

In [73]:
print('Enter the range of the years which you want the number of deaths to be calculated and shown')
start_year = int(input('Enter the desired start year: '))
end_year= int(input('Enter the desired end year: '))

Enter the range of the years which you want the number of deaths to be calculated and shown
Enter the desired start year: 2012
Enter the desired end year: 2020


In [74]:
# Creating a summary table for All policies
all_policies = data.loc[(data['All Policies Indicator']==1) & (data['Year of Death']>=start_year)\
                       & (data['Year of Death'] <=end_year)]
summary_allpol = all_policies.pivot_table(index=['Age_band_index','Age_band'],columns='Year of Death',\
                                          values='No_of death',aggfunc='sum',fill_value=0)
summary_allpol

,Year of Death,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0
Age_band_index,Age_band,,,,,,,,,
1,0-25,0,16,7,18,16,17,12,14,10
2,26-30,0,9,4,6,7,15,15,11,13
3,31-35,0,15,23,21,15,29,31,26,24
4,36-40,0,38,31,22,23,46,34,26,31
5,41-45,0,38,37,26,35,29,39,38,49
6,46-50,0,35,31,30,27,37,47,21,28
7,51-55,0,23,33,36,40,45,30,24,45
8,Above 55,0,17,22,25,39,34,27,38,82


In [17]:
# Creating a summary table for male
male_df = data.loc[(data['Sex']=='M') & (data['Year of Death']>=start_year)\
                       & (data['Year of Death'] <=end_year)]
summary_male = male_df.pivot_table(index=['Age_band_index','Age_band'],columns='Year of Death',\
                                          values='No_of death',aggfunc='sum',fill_value=0)
summary_male

,Year of Death,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0
Age_band_index,Age_band,,,,,,,,,,
1,0-25,7,3,11,7,6,7,10,7,8,6
2,26-30,7,4,3,6,9,9,8,8,9,7
3,31-35,9,15,12,9,19,18,15,18,10,14
4,36-40,28,16,10,16,26,23,18,13,42,20
5,41-45,24,18,19,22,22,24,19,33,37,34
6,46-50,22,21,17,13,27,25,10,22,36,34
7,51-55,18,26,28,25,30,23,17,36,57,31
8,Above 55,14,17,20,21,22,18,34,64,76,40


In [18]:
# Creating a summary table for female
female_df = data.loc[(data['Sex']=='F') & (data['Year of Death']>=start_year)\
                       & (data['Year of Death'] <=end_year)]
summary_female = female_df.pivot_table(index=['Age_band_index','Age_band'],columns='Year of Death',\
                                          values='No_of death',aggfunc='sum',fill_value=0)
summary_female

,Year of Death,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0
Age_band_index,Age_band,,,,,,,,,,
1,0-25,9,4,7,9,11,5,4,3,7,8
2,26-30,2,0,3,1,6,6,3,5,2,5
3,31-35,6,8,9,6,10,13,11,6,16,15
4,36-40,10,15,12,7,20,11,8,18,35,21
5,41-45,14,19,7,13,7,15,19,16,15,22
6,46-50,13,10,13,14,10,22,11,6,15,14
7,51-55,5,7,8,15,15,7,7,9,23,28
8,Above 55,3,5,5,18,12,9,4,18,30,28


In [19]:
# Creating a summary table for Conventional Business type
conventional_df = data.loc[(data['Business Type']=='Conventional') & (data['Year of Death']>=
                                                                      start_year)\
                       & (data['Year of Death'] <=end_year)]
summary_conventional = conventional_df.pivot_table(index=['Age_band_index','Age_band'],columns='Year of Death',\
                                          values='No_of death',aggfunc='sum',fill_value=0)
summary_conventional

,Year of Death,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0
Age_band_index,Age_band,,,,,,,,,,
1,0-25,16,7,18,16,17,11,14,10,14,14
2,26-30,8,3,5,7,14,15,10,11,11,12
3,31-35,13,19,18,14,28,28,26,21,24,26
4,36-40,32,25,15,21,43,33,25,27,65,37
5,41-45,30,28,25,26,27,36,36,41,47,52
6,46-50,28,28,26,20,33,43,20,23,45,42
7,51-55,17,27,30,35,38,22,20,35,75,53
8,Above 55,10,15,17,23,25,21,31,55,71,55


In [20]:
# Creating a summary table for Unit linked business type
unit_linked_df = data.loc[(data['Business Type']=='Unit Linked') & (data['Year of Death']>=start_year)\
                       & (data['Year of Death'] <=end_year)]
summary_unit_linked = unit_linked_df.pivot_table(index=['Age_band_index','Age_band'],\
                                                 columns='Year of Death',\
                                          values='No_of death',aggfunc='sum',fill_value=0)
summary_unit_linked

,Year of Death,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0
Age_band_index,Age_band,,,,,,,,,,
1,0-25,0,0,0,0,0,1,0,0,1,0
2,26-30,1,1,1,0,1,0,1,2,0,0
3,31-35,2,4,3,1,1,3,0,3,2,3
4,36-40,6,6,7,2,3,1,1,4,12,4
5,41-45,8,9,1,9,2,3,2,8,5,4
6,46-50,7,3,4,7,4,4,1,5,6,6
7,51-55,6,6,6,5,7,8,4,10,5,6
8,Above 55,7,7,8,16,9,6,7,27,35,13


In [21]:
# Creating a summary table for Lien
lien_df = data.loc[(data['Lien/Non_Lien']=='Lien') & (data['Year of Death']>=start_year)\
                       & (data['Year of Death'] <=end_year)]
summary_lien = lien_df.pivot_table(index=['Age_band_index','Age_band'],columns='Year of Death',\
                                          values='No_of death',aggfunc='sum',fill_value=0)
summary_lien

,Year of Death,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0
Age_band_index,Age_band,,,,,,,,,,
1,0-25,2,3,11,12,10,10,6,6,12,12
2,26-30,8,3,2,6,9,13,3,7,8,9
3,31-35,8,12,15,10,19,19,18,16,16,15
4,36-40,24,16,11,14,25,22,19,17,42,23
5,41-45,14,20,12,15,18,23,21,32,30,24
6,46-50,16,14,15,12,26,20,12,16,29,25
7,51-55,8,11,11,19,16,13,11,16,32,18
8,Above 55,6,7,11,14,14,8,17,23,33,21


In [22]:
# Creating a summary table for Non_Lien
Non_Lien_df = data.loc[(data['Lien/Non_Lien']=='Non Lien') & (data['Year of Death']>=start_year)\
                       & (data['Year of Death'] <=end_year)]
summary_non_lien = Non_Lien_df.pivot_table(index=['Age_band_index','Age_band'],\
                                           columns='Year of Death',\
                                          values='No_of death',aggfunc='sum',fill_value=0)
summary_non_lien

,Year of Death,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0
Age_band_index,Age_band,,,,,,,,,,
1,0-25,14,4,7,4,7,2,8,4,3,2
2,26-30,1,1,4,1,6,2,8,6,3,3
3,31-35,7,11,6,5,10,12,8,8,10,14
4,36-40,14,15,11,9,21,12,7,14,35,18
5,41-45,24,17,14,20,11,16,17,17,22,32
6,46-50,19,17,15,15,11,27,9,12,22,23
7,51-55,15,22,25,21,29,17,13,29,48,41
8,Above 55,11,15,14,25,20,19,21,59,73,47


In [23]:
data.head()

,Policy No,Ben Ord No,Status,Plan No,Issue Age,Issue Date,Next Prem Date,Sex,Sum Assured,Std Prem,...,Exposure Contribution to Entry Age(in_years),Exposure Contribution to Exit Age(in_years),No_of death,male(1)/female(0)_indicator,Business Type,Lien/Non_Lien,All Policies Indicator,Total Premium,Age_band,Age_band_index
0,123-3,1,A,123,41,2003-08-02,2023-12-02,F,1000000.0,90996.00,...,0.579740,0.416153,0,0,Conventional,Non Lien,1,90996.00,Above 55,8
1,122-459,1,A,122,40,2004-01-01,2024-01-01,M,300000.0,29220.00,...,0.997947,0.997947,0,1,Conventional,Lien,1,29220.00,Above 55,8
2,122-631,1,A,122,30,2004-01-01,2024-01-01,M,300000.0,27528.00,...,0.996578,0.999316,0,1,Conventional,Lien,1,27528.00,46-50,6
3,120-137,1,A,120,32,2003-10-01,2024-02-01,F,350000.0,31008.00,...,0.746064,0.249829,0,0,Conventional,Lien,1,31008.00,51-55,7
4,121-159,1,A,121,33,2003-11-01,2024-02-01,F,320000.0,29776.16,...,0.828884,0.167009,0,0,Conventional,Lien,1,29776.16,51-55,7


In [24]:
# Next we will create age bands for age at entry years 
# This will be used when calculating the exposure summaries for all cohorts

# Converting the specific column to integer type
data['Age @entry-Years'] = data['Age @entry-Years'].astype(int)
data['Age @entry-Years'].dtype

dtype('int32')

In [25]:
# Creating the agebands for age @ entry years
data['Age_band_2'] = data['Age @entry-Years'].apply(create_ageband)

In [26]:
# Creating the ageband indices
data['Age_band2_index'] = data['Age_band_2'].apply(ageband_index)
# Taking a look at the resulting dataframe
data.head()

,Policy No,Ben Ord No,Status,Plan No,Issue Age,Issue Date,Next Prem Date,Sex,Sum Assured,Std Prem,...,No_of death,male(1)/female(0)_indicator,Business Type,Lien/Non_Lien,All Policies Indicator,Total Premium,Age_band,Age_band_index,Age_band_2,Age_band2_index
0,123-3,1,A,123,41,2003-08-02,2023-12-02,F,1000000.0,90996.00,...,0,0,Conventional,Non Lien,1,90996.00,Above 55,8,46-50,6
1,122-459,1,A,122,40,2004-01-01,2024-01-01,M,300000.0,29220.00,...,0,1,Conventional,Lien,1,29220.00,Above 55,8,46-50,6
2,122-631,1,A,122,30,2004-01-01,2024-01-01,M,300000.0,27528.00,...,0,1,Conventional,Lien,1,27528.00,46-50,6,36-40,4
3,120-137,1,A,120,32,2003-10-01,2024-02-01,F,350000.0,31008.00,...,0,0,Conventional,Lien,1,31008.00,51-55,7,41-45,5
4,121-159,1,A,121,33,2003-11-01,2024-02-01,F,320000.0,29776.16,...,0,0,Conventional,Lien,1,29776.16,51-55,7,41-45,5


In [27]:
# Calculating the total exposure contribution (for both exit and entry)
data['Total_contribution'] = data['Exposure Contribution to Entry Age(in_years)'] + data['Exposure Contribution to Exit Age(in_years)']
data.head()

,Policy No,Ben Ord No,Status,Plan No,Issue Age,Issue Date,Next Prem Date,Sex,Sum Assured,Std Prem,...,male(1)/female(0)_indicator,Business Type,Lien/Non_Lien,All Policies Indicator,Total Premium,Age_band,Age_band_index,Age_band_2,Age_band2_index,Total_contribution
0,123-3,1,A,123,41,2003-08-02,2023-12-02,F,1000000.0,90996.00,...,0,Conventional,Non Lien,1,90996.00,Above 55,8,46-50,6,0.995893
1,122-459,1,A,122,40,2004-01-01,2024-01-01,M,300000.0,29220.00,...,1,Conventional,Lien,1,29220.00,Above 55,8,46-50,6,1.995893
2,122-631,1,A,122,30,2004-01-01,2024-01-01,M,300000.0,27528.00,...,1,Conventional,Lien,1,27528.00,46-50,6,36-40,4,1.995893
3,120-137,1,A,120,32,2003-10-01,2024-02-01,F,350000.0,31008.00,...,0,Conventional,Lien,1,31008.00,51-55,7,41-45,5,0.995893
4,121-159,1,A,121,33,2003-11-01,2024-02-01,F,320000.0,29776.16,...,0,Conventional,Lien,1,29776.16,51-55,7,41-45,5,0.995893


In [28]:
# Calculating the cumulative sum for the respective age band and Issue Year groups.
# This will be for the all_policies category.
data_allp = data
data_allp['Cumulative_sum'] = data_allp.groupby(['Age_band_2','Issue Year'])['Total_contribution'].cumsum()
# Pivoting the dataFrame to get desired table
allp_pivot_df = data_allp.pivot_table(index=['Age_band2_index','Age_band_2'], columns='Issue Year',\
                                      values='Cumulative_sum', aggfunc='last',fill_value=0)

# Calculating the cumaltive sum row_wise
allp_pivot_df = allp_pivot_df.cumsum(axis=1)
# Shifting the values one column to the right
allp_pivot_df = allp_pivot_df.shift(axis=1)  
# Finding the index of the column 2013
start_column = allp_pivot_df.columns.get_loc(2013)

# Slicing the dataFrame to keep columns from Issue Year 2013 onwards
filtered_pivot_df = allp_pivot_df.iloc[:, start_column:]
# Rounding the values to remain with whole numbers
e_summary_allpol = filtered_pivot_df.round(0)
# Taking a look at the resulting summary
e_summary_allpol

,Issue Year,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Age_band2_index,Age_band_2,,,,,,,,,,
1,0-25,4860.0,7210.0,9167.0,11544.0,14692.0,17306.0,20062.0,23563.0,25742.0,28143.0
2,26-30,5306.0,10295.0,16160.0,22289.0,29736.0,36421.0,42987.0,49857.0,54919.0,60300.0
3,31-35,10021.0,16455.0,24161.0,32120.0,40744.0,48630.0,56652.0,65326.0,71735.0,78260.0
4,36-40,11688.0,16788.0,22786.0,29061.0,35838.0,42024.0,48708.0,55946.0,61467.0,67002.0
5,41-45,11286.0,14229.0,17898.0,21730.0,25940.0,29848.0,34394.0,39482.0,43289.0,47083.0
6,46-50,8706.0,10333.0,12331.0,14450.0,16956.0,19108.0,21894.0,25083.0,27549.0,30209.0
7,51-55,5231.0,6137.0,7289.0,8385.0,9762.0,11024.0,12841.0,14906.0,16622.0,18339.0
8,Above 55,2539.0,2882.0,3364.0,3866.0,4549.0,5354.0,6726.0,8268.0,9553.0,10903.0


In [29]:
# Calculating the cumulative sum for the respective age band and Issue Year groups.
# This will be for the male category.
data_male = data[data['male(1)/female(0)_indicator']==1]
data_male['Cumulative_sum'] = data_male.groupby(['Age_band_2','Issue Year'])['Total_contribution'].cumsum()
# Pivoting the dataFrame to get desired table
male_pivot_df = data_male.pivot_table(index=['Age_band2_index','Age_band_2'], columns='Issue Year',\
                                      values='Cumulative_sum', aggfunc='last',fill_value=0)

# Calculating the cumaltive sum row_wise
male_pivot_df = male_pivot_df.cumsum(axis=1)
# Shifting the values one column to the right
male_pivot_df = male_pivot_df.shift(axis=1)  
# Finding the index of the column 2013
start_column = male_pivot_df.columns.get_loc(2013)

# Slicing the dataFrame to keep columns from Issue Year 2013 onwards
filtered_pivot_df = male_pivot_df.iloc[:, start_column:]
# Rounding the values to remain with whole numbers
e_summary_male = filtered_pivot_df.round(0)
# Taking a look at the resulting summary
e_summary_male

C:\Users\hp\AppData\Local\Temp\ipykernel_21188\1294024638.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_male['Cumulative_sum'] = data_male.groupby(['Age_band_2','Issue Year'])['Total_contribution'].cumsum()


,Issue Year,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Age_band2_index,Age_band_2,,,,,,,,,,
1,0-25,2465.0,3665.0,4667.0,5877.0,7595.0,9010.0,10468.0,12299.0,13405.0,14619.0
2,26-30,2535.0,4816.0,7658.0,10600.0,14387.0,17579.0,20753.0,23992.0,26301.0,28831.0
3,31-35,5188.0,8479.0,12611.0,16817.0,21379.0,25399.0,29244.0,33528.0,36489.0,39494.0
4,36-40,6418.0,9380.0,12834.0,16418.0,20310.0,23511.0,26825.0,30427.0,33122.0,35730.0
5,41-45,6592.0,8333.0,10549.0,12854.0,15446.0,17618.0,19959.0,22654.0,24507.0,26416.0
6,46-50,5368.0,6352.0,7567.0,8894.0,10383.0,11585.0,13113.0,14855.0,16188.0,17570.0
7,51-55,3316.0,3855.0,4576.0,5280.0,6132.0,6857.0,7786.0,8872.0,9810.0,10708.0
8,Above 55,1652.0,1866.0,2162.0,2449.0,2833.0,3284.0,3992.0,4813.0,5474.0,6119.0


In [30]:
data['Business Type'].unique()

array(['Conventional', 'Unit Linked'], dtype=object)

In [31]:
# Calculating the cumulative sum for the respective age band and Issue Year groups.
# This will be for the female category.
data_female = data[data['male(1)/female(0)_indicator']==0]
data_female['Cumulative_sum'] = data_female.groupby(['Age_band_2','Issue Year'])['Total_contribution'].cumsum()
# Pivoting the dataFrame to get desired table
female_pivot_df = data_female.pivot_table(index=['Age_band2_index','Age_band_2'], columns='Issue Year',\
                                      values='Cumulative_sum', aggfunc='last',fill_value=0)

# Calculating the cumaltive sum row_wise
female_pivot_df = female_pivot_df.cumsum(axis=1)
# Shifting the values one column to the right
female_pivot_df = female_pivot_df.shift(axis=1)  
# Finding the index of the column 2013
start_column = female_pivot_df.columns.get_loc(2013)

# Slicing the dataFrame to keep columns from Issue Year 2013 onwards
filtered_pivot_df = female_pivot_df.iloc[:, start_column:]
# Rounding the values to remain with whole numbers
e_summary_female = filtered_pivot_df.round(0)
# Taking a look at the resulting summary
e_summary_female

C:\Users\hp\AppData\Local\Temp\ipykernel_21188\3014773352.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_female['Cumulative_sum'] = data_female.groupby(['Age_band_2','Issue Year'])['Total_contribution'].cumsum()


,Issue Year,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Age_band2_index,Age_band_2,,,,,,,,,,
1,0-25,2396.0,3545.0,4499.0,5667.0,7097.0,8297.0,9594.0,11264.0,12337.0,13524.0
2,26-30,2771.0,5479.0,8502.0,11689.0,15349.0,18842.0,22234.0,25865.0,28619.0,31469.0
3,31-35,4832.0,7976.0,11550.0,15303.0,19364.0,23231.0,27408.0,31798.0,35246.0,38766.0
4,36-40,5270.0,7408.0,9953.0,12643.0,15528.0,18513.0,21883.0,25520.0,28345.0,31272.0
5,41-45,4694.0,5897.0,7350.0,8876.0,10494.0,12230.0,14435.0,16828.0,18782.0,20667.0
6,46-50,3337.0,3980.0,4764.0,5556.0,6573.0,7523.0,8782.0,10228.0,11361.0,12640.0
7,51-55,1915.0,2282.0,2712.0,3105.0,3630.0,4167.0,5055.0,6034.0,6812.0,7631.0
8,Above 55,887.0,1016.0,1202.0,1418.0,1717.0,2069.0,2733.0,3455.0,4079.0,4784.0


In [32]:
# Calculating the cumulative sum for the respective age band and Issue Year groups.
# This will be for the Conventional Business type.
data_conventional = data[data['Business Type']=='Conventional']
data_conventional['Cumulative_sum'] = data_conventional.groupby(['Age_band_2','Issue Year'])['Total_contribution'].cumsum()
# Pivoting the dataFrame to get desired table
conventional_pivot_df = data_conventional.pivot_table(index=['Age_band2_index','Age_band_2'], columns='Issue Year',\
                                      values='Cumulative_sum', aggfunc='last',fill_value=0)

# Calculating the cumaltive sum row_wise
conventional_pivot_df = conventional_pivot_df.cumsum(axis=1)
# Shifting the values one column to the right
conventional_pivot_df = conventional_pivot_df.shift(axis=1)  
# Finding the index of the column 2013
start_column = conventional_pivot_df.columns.get_loc(2013)

# Slicing the dataFrame to keep columns from Issue Year 2013 onwards
filtered_pivot_df = conventional_pivot_df.iloc[:, start_column:]
# Rounding the values to remain with whole numbers
e_summary_conventional = filtered_pivot_df.round(0)
# Taking a look at the resulting summary
e_summary_conventional

C:\Users\hp\AppData\Local\Temp\ipykernel_21188\127195124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_conventional['Cumulative_sum'] = data_conventional.groupby(['Age_band_2','Issue Year'])['Total_contribution'].cumsum()


,Issue Year,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Age_band2_index,Age_band_2,,,,,,,,,,
1,0-25,4757.0,7092.0,9048.0,11425.0,14573.0,17154.0,19726.0,22447.0,24203.0,26209.0
2,26-30,4193.0,9152.0,15016.0,21145.0,28592.0,35175.0,41419.0,47758.0,52341.0,57248.0
3,31-35,7097.0,13487.0,21193.0,29152.0,37775.0,45540.0,53224.0,61392.0,67289.0,73416.0
4,36-40,8191.0,13267.0,19266.0,25540.0,32318.0,38371.0,44753.0,51550.0,56601.0,61745.0
5,41-45,8345.0,11253.0,14922.0,18754.0,22964.0,26787.0,31046.0,35727.0,39144.0,42582.0
6,46-50,6574.0,8169.0,10168.0,12286.0,14792.0,16876.0,19453.0,22363.0,24516.0,26922.0
7,51-55,3731.0,4600.0,5751.0,6847.0,8224.0,9443.0,11084.0,12949.0,14435.0,15928.0
8,Above 55,1488.0,1791.0,2273.0,2776.0,3459.0,4103.0,5204.0,6377.0,7266.0,8200.0


In [33]:
# Calculating the cumulative sum for the respective age band and Issue Year groups.
# This will be for the Unit Linked Business type.

data_unit_linked = data[data['Business Type']=='Unit Linked'].fillna(0)
data_unit_linked['Cumulative_sum'] = data_unit_linked.groupby(['Age_band_2','Issue Year'])['Total_contribution'].cumsum()
# Pivoting the dataFrame to get desired table
unit_linked_pivot_df = data_unit_linked.pivot_table(index=['Age_band2_index','Age_band_2'], columns='Issue Year',\
                                      values='Cumulative_sum', aggfunc='last',fill_value=0)

# Calculating the cumaltive sum row_wise
unit_linked_pivot_df = unit_linked_pivot_df.cumsum(axis=1)
# Shifting the values one column to the right
unit_linked_pivot_df = unit_linked_pivot_df.shift(axis=1)  
# Finding the index of the column 2013
start_column = unit_linked_pivot_df.columns.get_loc(2013)

# Slicing the dataFrame to keep columns from Issue Year 2013 onwards
filtered_pivot_df = unit_linked_pivot_df.iloc[:, start_column:]
# Rounding the values to remain with whole numbers
e_summary_unit_linked = filtered_pivot_df.round(0)
# Taking a look at the resulting summary
e_summary_unit_linked

,Issue Year,2013,2017,2018,2019,2020,2021,2022
Age_band2_index,Age_band_2,,,,,,,
1,0-25,104.0,119.0,153.0,336.0,1116.0,1539.0,1935.0
2,26-30,1114.0,1144.0,1246.0,1568.0,2099.0,2578.0,3052.0
3,31-35,2923.0,2969.0,3090.0,3428.0,3934.0,4446.0,4844.0
4,36-40,3497.0,3521.0,3653.0,3955.0,4397.0,4866.0,5258.0
5,41-45,2941.0,2976.0,3061.0,3348.0,3755.0,4145.0,4501.0
6,46-50,2132.0,2163.0,2232.0,2442.0,2720.0,3032.0,3287.0
7,51-55,1500.0,1538.0,1581.0,1757.0,1957.0,2188.0,2411.0
8,Above 55,1051.0,1091.0,1250.0,1521.0,1891.0,2287.0,2703.0


In [34]:
# Calculating the cumulative sum for the respective age band and Issue Year groups.
# This will be for the Lien type policy.
data_lien = data[data['Lien/Non_Lien']=='Lien']
data_lien['Cumulative_sum'] = data_lien.groupby(['Age_band_2','Issue Year'])['Total_contribution'].cumsum()
# Pivoting the dataFrame to get desired table
lien_pivot_df = data_lien.pivot_table(index=['Age_band2_index','Age_band_2'], columns='Issue Year',\
                                      values='Cumulative_sum', aggfunc='last',fill_value=0)

# Calculating the cumaltive sum row_wise
lien_pivot_df = lien_pivot_df.cumsum(axis=1)
# Shifting the values one column to the right
lien_pivot_df = lien_pivot_df.shift(axis=1)  
# Finding the index of the column 2013
start_column = lien_pivot_df.columns.get_loc(2013)

# Slicing the dataFrame to keep columns from Issue Year 2013 onwards
filtered_pivot_df = lien_pivot_df.iloc[:, start_column:]
# Rounding the values to remain with whole numbers
e_summary_lien = filtered_pivot_df.round(0)
# Taking a look at the resulting summary
e_summary_lien

C:\Users\hp\AppData\Local\Temp\ipykernel_21188\602609764.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_lien['Cumulative_sum'] = data_lien.groupby(['Age_band_2','Issue Year'])['Total_contribution'].cumsum()


,Issue Year,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Age_band2_index,Age_band_2,,,,,,,,,,
1,0-25,2111.0,4019.0,5455.0,7216.0,9498.0,11342.0,12889.0,14392.0,15410.0,16510.0
2,26-30,3412.0,7424.0,11548.0,16179.0,21737.0,26344.0,30123.0,33796.0,36398.0,39117.0
3,31-35,5415.0,10263.0,15262.0,21145.0,27281.0,32647.0,37265.0,41814.0,44981.0,48247.0
4,36-40,5243.0,9152.0,12851.0,17287.0,21837.0,25640.0,29038.0,32395.0,34795.0,37136.0
5,41-45,3989.0,6241.0,8397.0,10953.0,13623.0,15958.0,17996.0,19942.0,21349.0,22800.0
6,46-50,2669.0,3903.0,4959.0,6265.0,7675.0,8746.0,9626.0,10535.0,11134.0,11809.0
7,51-55,1660.0,2288.0,2773.0,3302.0,3890.0,4313.0,4611.0,4904.0,5105.0,5320.0
8,Above 55,611.0,810.0,1104.0,1403.0,1727.0,1947.0,2064.0,2168.0,2228.0,2308.0


In [35]:
data['Lien/Non_Lien'].unique()

array(['Non Lien', 'Lien'], dtype=object)

In [36]:
# Calculating the cumulative sum for the respective age band and Issue Year groups.
# This will be for the Non_lien type policy.
data_non_lien = data[data['Lien/Non_Lien']=='Non Lien']
data_non_lien['Cumulative_sum'] = data_non_lien.groupby(['Age_band_2','Issue Year'])['Total_contribution'].cumsum()
# Pivoting the dataFrame to get desired table
non_lien_pivot_df = data_non_lien.pivot_table(index=['Age_band2_index','Age_band_2'], columns='Issue Year',\
                                      values='Cumulative_sum', aggfunc='last',fill_value=0)

# Calculating the cumaltive sum row_wise
non_lien_pivot_df = non_lien_pivot_df.cumsum(axis=1)
# Shifting the values one column to the right
non_lien_pivot_df = non_lien_pivot_df.shift(axis=1)  
# Finding the index of the column 2013
start_column = non_lien_pivot_df.columns.get_loc(2013)

# Slicing the dataFrame to keep columns from Issue Year 2013 onwards
filtered_pivot_df = non_lien_pivot_df.iloc[:, start_column:]
# Rounding the values to remain with whole numbers
e_summary_non_lien = filtered_pivot_df.round(0)
# Taking a look at the resulting summary
e_summary_non_lien

C:\Users\hp\AppData\Local\Temp\ipykernel_21188\694586125.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_non_lien['Cumulative_sum'] = data_non_lien.groupby(['Age_band_2','Issue Year'])['Total_contribution'].cumsum()


,Issue Year,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Age_band2_index,Age_band_2,,,,,,,,,,
1,0-25,2750.0,3191.0,3712.0,4328.0,5194.0,5965.0,7173.0,9172.0,10332.0,11634.0
2,26-30,1894.0,2871.0,4612.0,6110.0,7999.0,10077.0,12863.0,16060.0,18521.0,21183.0
3,31-35,4606.0,6192.0,8899.0,10975.0,13463.0,15983.0,19387.0,23512.0,26754.0,30013.0
4,36-40,6445.0,7636.0,9935.0,11773.0,14002.0,16384.0,19670.0,23551.0,26672.0,29866.0
5,41-45,7297.0,7989.0,9502.0,10777.0,12318.0,13890.0,16398.0,19539.0,21940.0,24283.0
6,46-50,6037.0,6429.0,7372.0,8185.0,9280.0,10362.0,12268.0,14548.0,16415.0,18401.0
7,51-55,3570.0,3849.0,4515.0,5082.0,5872.0,6711.0,8230.0,10002.0,11517.0,13019.0
8,Above 55,1928.0,2072.0,2260.0,2463.0,2822.0,3407.0,4661.0,6101.0,7325.0,8595.0


### Calculating mortality

In [37]:
mortality_allpol = (summary_allpol/e_summary_allpol)*1000
mortality_allpol

,Year of Death,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0
Age_band_index,Age_band,,,,,,,,,,
1,0-25,3.292181,0.970874,1.963565,1.386001,1.157092,0.693401,0.697837,0.424394,0.582705,0.497459
2,26-30,1.696193,0.388538,0.371287,0.314056,0.504439,0.411850,0.255891,0.260746,0.200295,0.199005
3,31-35,1.496857,1.397751,0.869169,0.466999,0.711761,0.637467,0.458942,0.367388,0.362445,0.370560
4,36-40,3.251198,1.846557,0.965505,0.791439,1.283554,0.809061,0.533793,0.554106,1.252705,0.611922
5,41-45,3.367003,2.600323,1.452676,1.610676,1.117965,1.306620,1.104844,1.241072,1.201229,1.189389
6,46-50,4.020216,3.000097,2.432893,1.868512,2.182118,2.459703,0.959167,1.116294,1.851247,1.588930
7,51-55,4.396865,5.377220,4.938949,4.770423,4.609711,2.721335,1.869013,3.018919,4.812899,3.217187
8,Above 55,6.695549,7.633588,7.431629,10.087946,7.474170,5.042959,5.649718,9.917755,11.095991,6.236816


In [38]:
mortality_male = (summary_male/e_summary_male)*1000
mortality_male

,Year of Death,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0
Age_band_index,Age_band,,,,,,,,,,
1,0-25,2.839757,0.818554,2.356975,1.191084,0.789993,0.776915,0.955292,0.569152,0.596792,0.410425
2,26-30,2.761341,0.830565,0.391747,0.566038,0.625565,0.511975,0.385486,0.333444,0.342192,0.242794
3,31-35,1.734773,1.769077,0.951550,0.535173,0.888723,0.708689,0.512926,0.536865,0.274055,0.354484
4,36-40,4.362730,1.705757,0.779180,0.974540,1.280158,0.978265,0.671016,0.427252,1.268039,0.559754
5,41-45,3.640777,2.160086,1.801119,1.711529,1.424317,1.362243,0.951952,1.456696,1.509773,1.287099
6,46-50,4.098361,3.306045,2.246597,1.461660,2.600405,2.157963,0.762602,1.480983,2.223870,1.935117
7,51-55,5.428227,6.744488,6.118881,4.734848,4.892368,3.354237,2.183406,4.057710,5.810398,2.895032
8,Above 55,8.474576,9.110397,9.250694,8.574929,7.765619,5.481121,8.517034,13.297320,13.883814,6.537016


In [39]:
mortality_female = (summary_female/e_summary_female)*1000
mortality_female

,Year of Death,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0
Age_band_index,Age_band,,,,,,,,,,
1,0-25,3.756260,1.128350,1.555901,1.588142,1.549951,0.602627,0.416927,0.266335,0.567399,0.591541
2,26-30,0.721761,0.000000,0.352858,0.085551,0.390905,0.318438,0.134928,0.193311,0.069884,0.158887
3,31-35,1.241722,1.003009,0.779221,0.392080,0.516422,0.559597,0.401343,0.188691,0.453952,0.386937
4,36-40,1.897533,2.024838,1.205667,0.553666,1.287996,0.594177,0.365581,0.705329,1.234786,0.671527
5,41-45,2.982531,3.221977,0.952381,1.464624,0.667048,1.226492,1.316245,0.950796,0.798637,1.064499
6,46-50,3.895715,2.512563,2.728799,2.519798,1.521375,2.924365,1.252562,0.586625,1.320306,1.107595
7,51-55,2.610966,3.067485,2.949853,4.830918,4.132231,1.679866,1.384768,1.491548,3.376395,3.669244
8,Above 55,3.382187,4.921260,4.159734,12.693935,6.988934,4.349928,1.463593,5.209841,7.354744,5.852843


In [40]:
mortality_conventional = (summary_conventional/e_summary_conventional)*1000
mortality_conventional

,Year of Death,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0
Age_band_index,Age_band,,,,,,,,,,
1,0-25,3.363464,0.987028,1.989390,1.400438,1.166541,0.641250,0.709723,0.445494,0.578441,0.534168
2,26-30,1.907942,0.327797,0.332978,0.331048,0.489647,0.426439,0.241435,0.230328,0.210160,0.209614
3,31-35,1.831760,1.408764,0.849337,0.480241,0.741231,0.614844,0.488501,0.342064,0.356670,0.354146
4,36-40,3.906727,1.884375,0.778574,0.822240,1.330528,0.860024,0.558622,0.523763,1.148390,0.599239
5,41-45,3.594967,2.488225,1.675379,1.386371,1.175753,1.343935,1.159570,1.147591,1.200695,1.221173
6,46-50,4.259203,3.427592,2.557042,1.627869,2.230936,2.547997,1.028119,1.028485,1.835536,1.560062
7,51-55,4.556419,5.869565,5.216484,5.111728,4.620623,2.329768,1.804403,2.702911,5.195705,3.327474
8,Above 55,6.720430,8.375209,7.479103,8.285303,7.227522,5.118206,5.956956,8.624745,9.771539,6.707317


In [41]:
mortality_unit_linked = (summary_unit_linked/e_summary_unit_linked)*1000
mortality_unit_linked

,,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0
Age_band_index,Age_band,,,,,,,,,,
1,0-25,0.000000,NaN,NaN,NaN,0.000000,6.535948,0.000000,0.000000,0.649773,0.000000
2,26-30,0.897666,NaN,NaN,NaN,0.874126,0.000000,0.637755,0.952835,0.000000,0.000000
3,31-35,0.684229,NaN,NaN,NaN,0.336814,0.970874,0.000000,0.762583,0.449843,0.619323
4,36-40,1.715756,NaN,NaN,NaN,0.852031,0.273748,0.252845,0.909711,2.466091,0.760746
5,41-45,2.720163,NaN,NaN,NaN,0.672043,0.980072,0.597372,2.130493,1.206273,0.888691
6,46-50,3.283302,NaN,NaN,NaN,1.849283,1.792115,0.409500,1.838235,1.978892,1.825373
7,51-55,4.000000,NaN,NaN,NaN,4.551365,5.060089,2.276608,5.109862,2.285192,2.488594
8,Above 55,6.660324,NaN,NaN,NaN,8.249313,4.800000,4.602235,14.278160,15.303892,4.809471


In [42]:
mortality_lien = (summary_lien/e_summary_lien)*1000
mortality_lien

,Year of Death,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0
Age_band_index,Age_band,,,,,,,,,,
1,0-25,0.947418,0.746454,2.016499,1.662971,1.052853,0.881679,0.465513,0.416898,0.778715,0.726832
2,26-30,2.344666,0.404095,0.173190,0.370851,0.414041,0.493471,0.099592,0.207125,0.219792,0.230079
3,31-35,1.477378,1.169249,0.982833,0.472925,0.696455,0.581983,0.483027,0.382647,0.355706,0.310900
4,36-40,4.577532,1.748252,0.855965,0.809857,1.144846,0.858034,0.654315,0.524772,1.207070,0.619345
5,41-45,3.509652,3.204615,1.429082,1.369488,1.321295,1.441283,1.166926,1.604653,1.405218,1.052632
6,46-50,5.994755,3.586984,3.024803,1.915403,3.387622,2.286760,1.246624,1.518747,2.604634,2.117029
7,51-55,4.819277,4.807692,3.966823,5.754088,4.113111,3.014143,2.385600,3.262643,6.268364,3.383459
8,Above 55,9.819967,8.641975,9.963768,9.978617,8.106543,4.108885,8.236434,10.608856,14.811490,9.098787


In [43]:
mortality_non_lien = (summary_non_lien/e_summary_non_lien)*1000
mortality_non_lien

,Year of Death,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0
Age_band_index,Age_band,,,,,,,,,,
1,0-25,5.090909,1.253526,1.885776,0.924214,1.347709,0.335289,1.115293,0.436110,0.290360,0.171910
2,26-30,0.527983,0.348311,0.867303,0.163666,0.750094,0.198472,0.621939,0.373599,0.161978,0.141623
3,31-35,1.519757,1.776486,0.674233,0.455581,0.742776,0.750798,0.412648,0.340252,0.373776,0.466465
4,36-40,2.172227,1.964379,1.107197,0.764461,1.499786,0.732422,0.355872,0.594455,1.312238,0.602692
5,41-45,3.289023,2.127926,1.473374,1.855804,0.893002,1.151908,1.036712,0.870055,1.002735,1.317794
6,46-50,3.147259,2.644268,2.034726,1.832621,1.185345,2.605675,0.733616,0.824856,1.340238,1.249932
7,51-55,4.201681,5.715770,5.537099,4.132231,4.938692,2.533155,1.579587,2.899420,4.167752,3.149243
8,Above 55,5.705394,7.239382,6.194690,10.150223,7.087172,5.576754,4.505471,9.670546,9.965870,5.468296


### Crude mortality

In [44]:
# Creating a summary table for All policies up to sixty years
all_policies = data.loc[(data['All Policies Indicator']==1) & (data['Year of Death']>=2013)\
                       & (data['Year of Death'] <=2022) & (data['Age @exit-Years'] <= 60)]
all_policies.head()
# Incase the totals are needed this will be used
tsummary_allpol = all_policies.pivot_table(index=['Age_band_index','Age_band'],columns='Year of Death',\
                                          values='No_of death',aggfunc='sum',fill_value=0)
# Calculating the row-wise total
tsummary_allpol['Total_C'] = tsummary_allpol.sum(axis=1)

# Calculate the total percentage for each row
total_percentage = (tsummary_allpol['Total_C'] / tsummary_allpol['Total_C'].sum()) * 100

# Add the total percentage column to the pivot table
tsummary_allpol['Total_Percentage'] = total_percentage
# Calculating the column-wise total
tsummary_allpol.loc['Total'] = tsummary_allpol.sum()


tsummary_allpol


Year of Death,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,Total_C,Total_Percentage
"(1, 0-25)",16.0,7.0,18.0,16.0,17.0,12.0,14.0,10.0,15.0,14.0,139.0,6.080490
"(2, 26-30)",9.0,4.0,6.0,7.0,15.0,15.0,11.0,13.0,11.0,12.0,103.0,4.505687
"(3, 31-35)",15.0,23.0,21.0,15.0,29.0,31.0,26.0,24.0,26.0,29.0,239.0,10.454943
"(4, 36-40)",38.0,31.0,22.0,23.0,46.0,34.0,26.0,31.0,77.0,41.0,369.0,16.141732
"(5, 41-45)",38.0,37.0,26.0,35.0,29.0,39.0,38.0,49.0,52.0,56.0,399.0,17.454068
"(6, 46-50)",35.0,31.0,30.0,27.0,37.0,47.0,21.0,28.0,51.0,48.0,355.0,15.529309
"(7, 51-55)",23.0,33.0,36.0,40.0,45.0,30.0,24.0,45.0,80.0,59.0,415.0,18.153981
"(8, Above 55)",12.0,12.0,17.0,22.0,21.0,18.0,21.0,47.0,56.0,41.0,267.0,11.679790
Total,186.0,178.0,176.0,185.0,239.0,226.0,181.0,247.0,368.0,300.0,2286.0,100.000000


In [45]:
actual_mortality = tsummary_allpol['Total_C'].max()
actual_mortality

2286.0

Total number of deaths as per actual experience is 2286

In [46]:
data.columns

Index(['Policy No', 'Ben Ord No', 'Status', 'Plan No', 'Issue Age',
       'Issue Date', 'Next Prem Date', 'Sex', 'Sum Assured', 'Std Prem',
       'Extra Prem', 'Frequency', 'Fund Value', 'Maturity Date', 'Term', 'WOP',
       'Ben Code No', 'Accumulated Guarantee', 'Category', 'Issue Year',
       'Next Prem Year', 'Date of Birth', 'Date of Death', 'Year of Death',
       'Notice Date', 'Reporting Delay_Months', 'Entry Date', 'Exit Date',
       'Death Indicator', 'Age @entry-Years', 'Age @exit-Years',
       'Exposure Contribution to Entry Age(in_years)',
       'Exposure Contribution to Exit Age(in_years)', 'No_of death',
       'male(1)/female(0)_indicator', 'Business Type', 'Lien/Non_Lien',
       'All Policies Indicator', 'Total Premium', 'Age_band', 'Age_band_index',
       'Age_band_2', 'Age_band2_index', 'Total_contribution',
       'Cumulative_sum'],
      dtype='object')

In [47]:
# Calculating the row-wise total
e_summary_allpol['Total_C'] = e_summary_allpol.sum(axis=1)

# Calculate the total percentage for each row
total_percentage = (e_summary_allpol['Total_C'] / e_summary_allpol['Total_C'].sum()) * 100

# Add the total percentage column to the pivot table
e_summary_allpol['Total_Percentage'] = total_percentage
# Calculating the column-wise total
e_summary_allpol.loc['Total'] = e_summary_allpol.sum()
e_summary_allpol

Issue Year,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Total_C,Total_Percentage
"(1, 0-25)",4860.0,7210.0,9167.0,11544.0,14692.0,17306.0,20062.0,23563.0,25742.0,28143.0,162289.0,8.253484
"(2, 26-30)",5306.0,10295.0,16160.0,22289.0,29736.0,36421.0,42987.0,49857.0,54919.0,60300.0,328270.0,16.694731
"(3, 31-35)",10021.0,16455.0,24161.0,32120.0,40744.0,48630.0,56652.0,65326.0,71735.0,78260.0,444104.0,22.585667
"(4, 36-40)",11688.0,16788.0,22786.0,29061.0,35838.0,42024.0,48708.0,55946.0,61467.0,67002.0,391308.0,19.900636
"(5, 41-45)",11286.0,14229.0,17898.0,21730.0,25940.0,29848.0,34394.0,39482.0,43289.0,47083.0,285179.0,14.503265
"(6, 46-50)",8706.0,10333.0,12331.0,14450.0,16956.0,19108.0,21894.0,25083.0,27549.0,30209.0,186619.0,9.490828
"(7, 51-55)",5231.0,6137.0,7289.0,8385.0,9762.0,11024.0,12841.0,14906.0,16622.0,18339.0,110536.0,5.621497
"(8, Above 55)",2539.0,2882.0,3364.0,3866.0,4549.0,5354.0,6726.0,8268.0,9553.0,10903.0,58004.0,2.949892
Total,59637.0,84329.0,113156.0,143445.0,178217.0,209715.0,244264.0,282431.0,310876.0,340239.0,1966309.0,100.000000


In [48]:
data.columns

Index(['Policy No', 'Ben Ord No', 'Status', 'Plan No', 'Issue Age',
       'Issue Date', 'Next Prem Date', 'Sex', 'Sum Assured', 'Std Prem',
       'Extra Prem', 'Frequency', 'Fund Value', 'Maturity Date', 'Term', 'WOP',
       'Ben Code No', 'Accumulated Guarantee', 'Category', 'Issue Year',
       'Next Prem Year', 'Date of Birth', 'Date of Death', 'Year of Death',
       'Notice Date', 'Reporting Delay_Months', 'Entry Date', 'Exit Date',
       'Death Indicator', 'Age @entry-Years', 'Age @exit-Years',
       'Exposure Contribution to Entry Age(in_years)',
       'Exposure Contribution to Exit Age(in_years)', 'No_of death',
       'male(1)/female(0)_indicator', 'Business Type', 'Lien/Non_Lien',
       'All Policies Indicator', 'Total Premium', 'Age_band', 'Age_band_index',
       'Age_band_2', 'Age_band2_index', 'Total_contribution',
       'Cumulative_sum'],
      dtype='object')

In [49]:
# Calculating the cumulative sum for the respective age at entry and Issue Year groups.
# This will be for the all_policies category.
data_age_allp = data[data['Age @entry-Years'] >= 0]
data_age_allp['Cumulative_sum'] = data_age_allp.groupby(['Age @entry-Years','Issue Year'])['Total_contribution'].cumsum()
# Pivoting the dataFrame to get desired table
age_allp_pivot_df = data_age_allp.pivot_table(index=['Age @entry-Years'], columns='Issue Year',\
                                      values='Cumulative_sum', aggfunc='last',fill_value=0)

# Calculating the cumaltive sum row_wise
age_allp_pivot_df = age_allp_pivot_df.cumsum(axis=1)
# Shifting the values one column to the right
age_allp_pivot_df = age_allp_pivot_df.shift(axis=1)  
# Finding the index of the column 2013
start_column = age_allp_pivot_df.columns.get_loc(2013)

# Slicing the dataFrame to keep columns from Issue Year 2013 onwards
filtered_pivot_df = age_allp_pivot_df.iloc[:, start_column:]
# Rounding the values to remain with whole numbers
e_age_summary_allpol = filtered_pivot_df.round(0)
# Taking a look at the resulting summary
e_age_summary_allpol

C:\Users\hp\AppData\Local\Temp\ipykernel_21188\1481613561.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_age_allp['Cumulative_sum'] = data_age_allp.groupby(['Age @entry-Years','Issue Year'])['Total_contribution'].cumsum()


Issue Year,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Age @entry-Years,,,,,,,,,,
0,0.0,9.0,9.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0
1,129.0,309.0,398.0,509.0,612.0,676.0,722.0,760.0,789.0,831.0
2,247.0,387.0,466.0,599.0,671.0,752.0,809.0,848.0,883.0,918.0
3,293.0,417.0,469.0,553.0,606.0,657.0,696.0,733.0,754.0,785.0
4,273.0,365.0,427.0,505.0,560.0,602.0,644.0,673.0,699.0,728.0
...,...,...,...,...,...,...,...,...,...,...
74,2.0,2.0,2.0,2.0,2.0,2.0,5.0,12.0,13.0,19.0
75,0.0,0.0,0.0,0.0,0.0,3.0,3.0,9.0,15.0,21.0
76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,11.0,15.0


In [50]:
# Calculating the row-wise total
e_age_summary_allpol['Total_C'] = e_age_summary_allpol.sum(axis=1)
e_age_summary_allpol.head()

Issue Year,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Total_C
Age @entry-Years,,,,,,,,,,,
0,0.0,9.0,9.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,95.0
1,129.0,309.0,398.0,509.0,612.0,676.0,722.0,760.0,789.0,831.0,5735.0
2,247.0,387.0,466.0,599.0,671.0,752.0,809.0,848.0,883.0,918.0,6580.0
3,293.0,417.0,469.0,553.0,606.0,657.0,696.0,733.0,754.0,785.0,5963.0
4,273.0,365.0,427.0,505.0,560.0,602.0,644.0,673.0,699.0,728.0,5476.0


In [51]:
# Merging the pivot table with the expected mortality dataframe using their index.
merged_table = e_age_summary_allpol.merge(expected_mortality_df, left_index=True, right_index=True)
merged_table

,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Total_C,Age Last Birthday,KE 01-03,KE 07-10
0,0.0,9.0,9.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,95.0,0,3.121,2.3005
1,129.0,309.0,398.0,509.0,612.0,676.0,722.0,760.0,789.0,831.0,5735.0,1,3.121,2.3005
2,247.0,387.0,466.0,599.0,671.0,752.0,809.0,848.0,883.0,918.0,6580.0,2,3.121,2.3005
3,293.0,417.0,469.0,553.0,606.0,657.0,696.0,733.0,754.0,785.0,5963.0,3,3.121,2.3005
4,273.0,365.0,427.0,505.0,560.0,602.0,644.0,673.0,699.0,728.0,5476.0,4,3.121,2.3005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,2.0,2.0,2.0,2.0,2.0,2.0,5.0,12.0,13.0,19.0,61.0,74,70.000,34.9220
75,0.0,0.0,0.0,0.0,0.0,3.0,3.0,9.0,15.0,21.0,51.0,75,76.500,39.2440
76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,11.0,15.0,34.0,76,83.400,44.0890
77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77,91.000,49.5160


In [52]:
# Calculating the expected deaths using KE01-03 
merged_table['Exp_deaths_KE_01-03'] =(merged_table['KE 01-03'] * merged_table['Total_C'])/1000
merged_table.head()

,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Total_C,Age Last Birthday,KE 01-03,KE 07-10,Exp_deaths_KE_01-03
0,0.0,9.0,9.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,95.0,0,3.121,2.3005,0.296495
1,129.0,309.0,398.0,509.0,612.0,676.0,722.0,760.0,789.0,831.0,5735.0,1,3.121,2.3005,17.898935
2,247.0,387.0,466.0,599.0,671.0,752.0,809.0,848.0,883.0,918.0,6580.0,2,3.121,2.3005,20.536180
3,293.0,417.0,469.0,553.0,606.0,657.0,696.0,733.0,754.0,785.0,5963.0,3,3.121,2.3005,18.610523
4,273.0,365.0,427.0,505.0,560.0,602.0,644.0,673.0,699.0,728.0,5476.0,4,3.121,2.3005,17.090596


### Expected deaths for KE_01-03

In [65]:
# Expected deaths for KE_01-03
merged_table_60 = merged_table.iloc[:61]
x = merged_table_60['Exp_deaths_KE_01-03'].sum()
x

5619.488638999998

Expected deaths for KE_01-03 is 5619.49

### Expected deaths for KE_07-10

In [66]:
# Calculating the expected deaths using KE_07-10 
merged_table['Exp_deaths_KE_07-10'] =(merged_table['KE 07-10'] * merged_table['Total_C'])/1000
merged_table.head()

,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Total_C,Age Last Birthday,KE 01-03,KE 07-10,Exp_deaths_KE_01-03,Exp_deaths_KE_07-10
0,0.0,9.0,9.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,95.0,0,3.121,2.3005,0.296495,0.218547
1,129.0,309.0,398.0,509.0,612.0,676.0,722.0,760.0,789.0,831.0,5735.0,1,3.121,2.3005,17.898935,13.193368
2,247.0,387.0,466.0,599.0,671.0,752.0,809.0,848.0,883.0,918.0,6580.0,2,3.121,2.3005,20.536180,15.137290
3,293.0,417.0,469.0,553.0,606.0,657.0,696.0,733.0,754.0,785.0,5963.0,3,3.121,2.3005,18.610523,13.717881
4,273.0,365.0,427.0,505.0,560.0,602.0,644.0,673.0,699.0,728.0,5476.0,4,3.121,2.3005,17.090596,12.597538


In [67]:
# Expected deaths for KE_07-10
merged_table_60 = merged_table.iloc[:61]
y = merged_table_60['Exp_deaths_KE_07-10'].sum()
y

4548.440372999999

Expected deaths for KE_07-10 is 4548.44

### Comparing actual mortality to expected

In [68]:
comp_ke_01_03 = actual_mortality/x
comp_ke_01_03

0.40679858023643933

In [70]:
comp_ke_07_10 = actual_mortality/y
comp_ke_07_10

0.5025898577389134